# Continuous Integration

As we're writing code for our package (a process that can take months/years and involve scores of developers), we always want the assurance that the package is not "broken" due to a recent innocent-looking commit or pull request.

While it's possible for us to write good tests, keep running `pytest` locally every time we make a change, we would like to be able to automate it, and enforce it for certain situations ("all tests must pass if we merge any code to the master branch or release a new version") for all developers working on the code.

The solution is referred to as *Continuous Integration* - a workflow to run checks-and-balances on our code as it is being written and pushed to our repository (so changes integrate continuously). The exact steps thus depend on where we're hosting our code. Assuming it's GitHub, we have a few choices:

- [GitHub Actions](https://docs.github.com/en/actions) (supports testing on Linux/MacOS/Windows)
- [Azure Pipelines](https://azure.microsoft.com/en-us/services/devops/pipelines/) (Linux/MacOS/Windows)
- [Travis CI](https://travis-ci.org/)  (MacOS/Windows)
- [Appveyor](https://www.appveyor.com/) (Windows)

GitHub Actions is the easiest to set up and use, and supports all the major OS platforms, so we'll be using that in this tutorial.

## GitHub Actions

The basic idea of GitHub Actions, or any other CI platform you use, is essentially:
    
- Specify *when* we would like our workflow to run (the *trigger*)

  Push/Pull/PR?

- Specify the (independent) *jobs* (series of steps) that should run

    - Specify what OSes we would like to run our job on - the *Build Matrix*.

      The build matrix can also contain other variables, like python versions, C++ compiler versions etc.
  
    - Specify the steps to take for each configuration of our build matrix.
    
      These correspond to what one would otherwise have to do manually on the command line to run the job.
      
      Typical steps for a python package would include:
      
        - Check out our package code from the repository
        - Install Python
        - Install any python library we might need (`pytest` etc.)
        - Build the package
        - Run `pytest` against the built package
      
A job that runs successfully to completion is said to *pass*, otherwise it *fails*. Within a job, individual *steps* can produce useful output for us to inspect later on.

If all jobs run successfully, our CI passes. This is the green badge that you must have seen many times on GitHub repositories:

![Main](https://github.com/vineetbansal/walrus/actions/workflows/main.yml/badge.svg?branch=master)

### Example

[Here](https://raw.githubusercontent.com/vineetbansal/walrus/master/.github/workflows/main-simple.yml) is an example of a real-life workflow for our walrus project.

```
name: Main-Simple

on:
  push:
    branches: [ master, develop ]
  pull_request:
    branches: [ master, develop ]

jobs:

  build_and_test:

    strategy:
      matrix:
        python: [3.7, 3.8, 3.9]

    runs-on: ubuntu-latest 

    steps:
      - uses: actions/checkout@v2
        with:
          lfs: true
          submodules: recursive 

      - name: Setup Python
        uses: actions/setup-python@v2
        with:
          python-version: ${{ matrix.python }}

      - name: Install global python dependencies
        run: |
          python -m pip install --upgrade pip 
          python -m pip install --upgrade pytest 

      - name: Install our package
        run: |
          pip install .
 
      - name: Test with pytest
        run: |
          pytest test 
```

Notice that a *step* can be something that someone else wrote up - like [actions/setup-python@v2](https://github.com/actions/setup-python), an *action*. More actions are available at the [GitHub Marketplace](https://github.com/marketplace?type=actions). Usually we just have to know what an action does and how to call it, not *how* it works.

### A slightly modified example

[Here](https://raw.githubusercontent.com/vineetbansal/walrus/master/.github/workflows/main.yml) is a slightly modified workflow for GitHub actions.

```
name: Main 

on:
  push:
    branches: [ master, develop ]
  pull_request:
    branches: [ master, develop ]

jobs:

  build_and_test:

    strategy:
      matrix:
        os: [ubuntu-latest, macos-latest]
      fail-fast: false

    runs-on: ${{ matrix.os }}

    steps:
      - uses: actions/checkout@v2
        with:
          lfs: true
          submodules: recursive 

      - name: Setup pyenv
        uses: "gabrielfalcao/pyenv-action@v7"
        with:
          default: 3.6.8
          versions: 3.7.9, 3.8.6, 3.9.0

      - name: Install global pyenv dependencies
        run: |
          which python
          python --version
          python -m pip install --upgrade pip 
          python -m pip install --upgrade virtualenv
          python -m pip install --upgrade nox 

      - name: Warm up virtualenv cache
        run: |
          ./virtualenv_warmup.sh

      - name: Test with nox
        run: |
          python -m nox 
```

Notice a few things about our workflow above. Instead of following the earlier series of steps:

- For every python version we wish to test against
    - Install Python
    - Check out our package code from the repository
    - Build the package
    - Run tests against the built package

We're doing:

- Check out our package code from the repository
- Using **pyenv**, install our **default** python, along with a bunch of additional python versions
- On our default python, install essential stuff like `pip`/`virtualenv`/`nox` etc.
- Run **nox**
    - Go through the bunch of python versions, and
        - Build our package
        - Run `pytest` against the built package
        
        
This allows us to mirror closely what we might do ourselves to test out the code locally - use `pyenv` and `nox`, tools that do a lot of heavy lifting for us and can (and should) be used on our laptops.

The second approach is what we would recommend, as it makes moving to other CI providers relatively easy, because most of the moving parts are now being handled by the `nox` library. It might take a little more work to set up initially though (notice the `virtualenv_warmup.sh` script?)